<a href="https://colab.research.google.com/github/wtsyang/UserIntentPrediction/blob/BERT/BERT/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/UserIntentPrediction'

/content/drive/My Drive/UserIntentPrediction


In [3]:
!pip install pytorch_pretrained_bert
!pip install tools

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 92kB 3.9MB/s 
  Created wheel for tools: filename=tools-0.1.9-cp36-none-any.whl size=46760 sha256=97b092dcbd062b4641d19be0256ea0a45f249c117d82c0a78d78b03377aa40f7
  Stored in directory: /root/.cache/pip/wheels/87/67/9b/1ca7dcb0b9ebfdc23a00c85a0644abb6fb14f9159a0df8e067
  Created wheel for pytils: filename=pytils-0.3-cp36-none-any.whl size=40355 sha256=e928549d099217565e5eebb0ba821a9297c0a02ef5072de15e82a2b48e8a8f76
  Stored in directory: /root/.cache/pip/wheels/d6/f9/dc/4f07d8ee40d9cfca9973b3f4aeff99d0bb69900e5f3dffbf32
Successfully built tools pytils


In [109]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import pandas as pd
import nltk
import string
import numpy as np
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Torch Version:',torch.__version__)
print('NLTK Version:',nltk.__version__)

Torch Version: 1.4.0
NLTK Version: 3.2.5


In [0]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "data/"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'intentionClassification'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
max_seq_length = 512

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'
num_labels=12
CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"


### Load the Dataset

In [0]:
Train=pd.read_csv('data/Train_Preprocessing.csv').reset_index(drop=True)
Valid=pd.read_csv('data/Valid_Preprocessing.csv').reset_index(drop=True)
Test=pd.read_csv('data/Test_Preprocessing.csv').reset_index(drop=True)

### Create Folder


In [0]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)
if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

### Prepare the Dataset

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
nltk.download('punkt')
sentence_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmpu4owv_0q
100%|██████████| 213450/213450 [00:00<00:00, 5181134.81B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpu4owv_0q to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpu4owv_0q
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
class Process():
  def __init__(self):
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
    nltk.download('punkt')
    self.sentence_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

  def processing(self,text):
    # Tokenization
    tokenized_text=self.tokenizer.tokenize(text)
    assert len(tokenized_text) <= (max_seq_length - 2)
    tokenized_text = ["[CLS]"] + tokenized_text + ["[SEP]"]
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    
     # Sentence Embedding
    segments_ids = [0] * len(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokens_tensor,segments_tensors

## Model

In [11]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained(BERT_MODEL)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz not found in cache, downloading to /tmp/tmptdkyva2a
100%|██████████| 404400730/404400730 [00:05<00:00, 68846733.91B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmptdkyva2a to cache at /root/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmptdkyva2a
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at /root/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e5

In [0]:
model.to(device)

In [125]:
process=Process()

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
# Predict hidden states features for each layer
with torch.no_grad():
  
    dbName='Train'
    Target=Train.copy()
    for i in range(len(Target)):
      sentences=sentence_tokenizer.tokenize(Target.loc[i,'utterance'])
      token_vecs_cat = []
      for sentence in sentences:
        temp=process.processing(sentence)
        encoded_layers, _ = model(temp[0], temp[1])
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
        # `token_embeddings` is a [tokenSize x layerSize x featureSize] tensor.
        # For each token in the sentence...
        for token in token_embeddings:
            # `token` is a [12 x 768] tensor
            # Concatenate the vectors (that is, append them together) from the last 
            # four layers.
            # Each layer vector is 768 values, so `cat_vec` is length 3,072.
            cat_vec = token[-1]#torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
            # Use `cat_vec` to represent `token`.
            token_vecs_cat.append(list(cat_vec.cpu().numpy()))
      sentencesTensor=np.array(token_vecs_cat)
      utterenceID=Target.loc[i,'id']
      diaglogID=Target.loc[i,'diaglogID']
      np.save('BERT/vector/'+dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy',sentencesTensor)

In [0]:
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
# Predict hidden states features for each layer
with torch.no_grad():
  
    dbName='Valid'
    Target=Valid.copy()
    for i in range(len(Target)):
      sentences=sentence_tokenizer.tokenize(Target.loc[i,'utterance'])
      token_vecs_cat = []
      for sentence in sentences:
        temp=process.processing(sentence)
        encoded_layers, _ = model(temp[0], temp[1])
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
        # `token_embeddings` is a [tokenSize x layerSize x featureSize] tensor.
        # For each token in the sentence...
        for token in token_embeddings:
            # `token` is a [12 x 768] tensor
            # Concatenate the vectors (that is, append them together) from the last 
            # four layers.
            # Each layer vector is 768 values, so `cat_vec` is length 3,072.
            cat_vec = token[-1]#torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
            # Use `cat_vec` to represent `token`.
            token_vecs_cat.append(list(cat_vec.cpu().numpy()))
      sentencesTensor=np.array(token_vecs_cat)
      utterenceID=Target.loc[i,'id']
      diaglogID=Target.loc[i,'diaglogID']
      np.save('BERT/vector/'+dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy',sentencesTensor)

In [0]:
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
# Predict hidden states features for each layer
with torch.no_grad():
  
    dbName='Test'
    Target=Test.copy()
    for i in range(len(Target)):
      sentences=sentence_tokenizer.tokenize(Target.loc[i,'utterance'])
      token_vecs_cat = []
      for sentence in sentences:
        temp=process.processing(sentence)
        encoded_layers, _ = model(temp[0], temp[1])
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
        # `token_embeddings` is a [tokenSize x layerSize x featureSize] tensor.
        # For each token in the sentence...
        for token in token_embeddings:
            # `token` is a [12 x 768] tensor
            # Concatenate the vectors (that is, append them together) from the last 
            # four layers.
            # Each layer vector is 768 values, so `cat_vec` is length 3,072.
            cat_vec = token[-1]#torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
            # Use `cat_vec` to represent `token`.
            token_vecs_cat.append(list(cat_vec.cpu().numpy()))
      sentencesTensor=np.array(token_vecs_cat)
      utterenceID=Target.loc[i,'id']
      diaglogID=Target.loc[i,'diaglogID']
      np.save('BERT/vector/'+dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy',sentencesTensor)